In [18]:
import os
import sys
import glob
import random
import operator as op
import itertools as it
import scipy.stats as stats
from functools import reduce, partial
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from scipy.stats import beta
from mpl_toolkits.mplot3d import Axes3D
sns.set_context("notebook", font_scale=1.5)
%matplotlib inline
random.seed(12345)

**10 dimensional Gaussian Case**

**Same leapfrog function, but with some printing to figure out what is wrong**

In [70]:
def leapfrog(U, grad_U, current_q, stepsize=0.25, steps=25):
    q = current_q
    p = np.random.normal(0,1,q.size) # independent statndard normal variates
    current_p = p
    q_list=q
    p_list=p
    
    # Half step for momentum at the beginning
    p = p - stepsize*(grad_U(q)/2)
    
    # Alternate full steps for position and momentum
    for i in range(steps):
        
        # Make a full step for the position
        q = q + stepsize*p
        q_list = np.vstack((q_list,q))
        
        # Make a full step for the momentum, except at end of trajectory
        if (i!=steps-1):
            p = p - stepsize*grad_U(q)
            p_list = np.vstack((p_list,p))
    
    # Make a half step for momentum at the end
    p = p - stepsize*(grad_U(q)/2)
    p_list = np.vstack((p_list,p))
    
    # Negate momentum at end of trajectory to make the proposal symmetric
    p = -p
    
    # Evaluate potential and kinetic energies at start and end of trajectory
    current_U = U(current_q)
    current_K = sum(current_p**2)/2
    proposed_U = U(q)
    proposed_K = sum(p**2)/2
    
    print("current_U =",current_U,",proposed_U=",proposed_U,",current_K=",current_K,",proposed_K=",proposed_K,",prob_ratio=",np.exp(current_U-proposed_U+current_K-proposed_K))
    # Accept or reject the state at end of trajectory, returning either the postion at the end
    # of the trajectory or the initial position
    if (np.random.uniform(0,1) < np.exp(current_U-proposed_U+current_K-proposed_K)):
        print("accept")
        return(q,p_list,q_list) # Accept

    else:
        print("reject")
        return(current_q,p_list,q_list) # Reject

    

**Generating from a 10D gaussian with the corresponding covariance structure**

In [64]:
Sigma = np.diag(np.arange(10,101,10)/100)**2
Sigma

array([[ 0.01,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.04,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.09,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.16,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.36,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.49,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.64,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.81,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ]])

**10D gaussian distribution and its gradient**

In [65]:
def U(q):
    '''
    Returns -log p(q)
    '''
    from scipy.stats import multivariate_normal
    return (-np.log(multivariate_normal.pdf(q, mean=np.zeros(q.size), cov = Sigma)))

def grad_U(q):
    return(np.dot(q,Sigma))

**Run the leapfrog step multiple times, it rejects almost everytime**

The parameters stepsize and number of steps are tuned for this particular case

In [75]:
current_q = np.random.random(10)
q,p_list,q_list=leapfrog(U, grad_U, current_q, stepsize=0.12, steps=15)

current_U = 7.46657098285 ,proposed_U= 435.244962395 ,current_K= 5.45944178359 ,proposed_K= 2.7755366414 ,prob_ratio= 2.41997191384e-185
reject


**Same HMC function, ignore this one till we solve the leapfrog thing**

In [ ]:
def HMC(U, grad_U, current_q, stepsize=0.25, steps=25, max_iter= 20):
    current_q_list = current_q
    for i in range(max_iter):
        current_q,p_list,q_list = leapfrog(U, grad_U, current_q, stepsize=0.25, steps=20)
        current_q_list = np.vstack((current_q_list,current_q))
    return(current_q_list[:-1,:])